In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sys
import warnings
from itertools import cycle
warnings.filterwarnings("ignore")
np.random.seed(40)
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import pickle
#Calling the preprocessor pickle
with open('prepic.pkl', 'rb') as f:
    pre = pickle.load(f)

mlflow.set_tracking_uri('http://127.0.0.1:8000')

train = pd.read_csv(r"D:\mlopset\train_set_ml.csv")

test = pd.read_csv(r"D:\mlopset\test_set_ml.csv")

import xgboost as xg

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    return rmse, mae

train_x = train.drop(["status"], axis=1)
test_x = test.drop(["status"], axis=1)
train_y = train[["status"]]
test_y = test[["status"]]

train_x_tf = pre.transform(train_x)
test_x_tf = pre.transform(test_x)

# Define the experiment name
experiment_name = "loan_status_experiment"

# Check if the experiment exists
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    # If the experiment does not exist, create it
    mlflow.create_experiment(experiment_name)

# Set the experiment
mlflow.set_experiment(experiment_name)

# Defining the hyperparameters
eta_v = [0.01,0.03,0.1,0.2]
subsample_v = [0.5, .9, 1]
max_depth_v = [6,8,10]
gamma_v = [0.01,.1, .2, .3]
max_delta_step_v = [2,4,6,8]

for eta in eta_v:
    for subsample in subsample_v:
        for max_depth in max_depth_v:
            for gamma in gamma_v:
                for max_delta_step in max_delta_step_v:
                    with mlflow.start_run(run_name=f"eta_{eta}_subsample_{subsample}_max_depth_{max_depth}_gamma_{gamma}_max_delta_step{max_delta_step}"):
                        # Run XGB
                        xgb = xg.XGBClassifier(eta=eta, subsample=subsample, max_depth=max_depth, gamma=gamma, max_delta_step=max_delta_step, random_state=42)
                        xgb.fit(train_x_tf, train_y)
                        predicted_qualities = xgb.predict(test_x_tf)
                        (rmse, mae) = eval_metrics(test_y, predicted_qualities)
            
                        # Print out XGB model metrics
                        print(f"XGBoost Model (eta={eta:f}, subsample={subsample:f}, max_depth={max_depth:f}, gamma={gamma:f}, max_delta_step={max_delta_step:f}):")
                        print(f"  RMSE: {rmse}")
                        print(f"  MAE: {mae}")
            
                        # Infer model signature
                        predictions = xgb.predict(train_x_tf)
                        signature = infer_signature(train_x_tf, predictions)
            
                        # Log mlflow attributes for mlflow UI
                        mlflow.log_param("eta", eta)
                        mlflow.log_param("subsample", subsample)
                        mlflow.log_param("max_depth", max_depth)
                        mlflow.log_param("gamma", gamma)
                        mlflow.log_param("max_delta_step", max_delta_step)
                        mlflow.log_metric("rmse", rmse)
                        mlflow.log_metric("mae", mae)
                        mlflow.sklearn.log_model(xgb, "model", signature=signature)

# end current run
mlflow.end_run()

# Get experiment ID
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

# Search all runs in experiment_id
runs = mlflow.search_runs(experiment_ids=experiment_id)

# display runs dataframe
runs

XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=2.000000):
  RMSE: 0.5584155773160767
  MAE: 0.3118279569892473
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=4.000000):
  RMSE: 0.5584155773160767
  MAE: 0.3118279569892473
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=6.000000):
  RMSE: 0.5584155773160767
  MAE: 0.3118279569892473
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=8.000000):
  RMSE: 0.5584155773160767
  MAE: 0.3118279569892473
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.100000, max_delta_step=2.000000):
  RMSE: 0.5584155773160767
  MAE: 0.3118279569892473
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.100000, max_delta_step=4.000000):
  RMSE: 0.5584155773160767
  MAE: 0.3118279569892473
XGBoost Model (e

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.mae,params.max_delta_step,params.gamma,params.eta,params.subsample,params.max_depth,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.log-model.history,tags.mlflow.source.name
0,cf0b3d946aa44990b7f8e49ee9887c4d,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/cf0b3d946...,2024-01-14 09:29:36.919000+00:00,2024-01-14 09:29:40.588000+00:00,0.486373,0.236559,8,0.3,0.2,1,10,LOCAL,eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,admin,"[{""run_id"": ""cf0b3d946aa44990b7f8e49ee9887c4d""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
1,8524d199f0214b47aa871f15c955e1de,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/8524d199f...,2024-01-14 09:29:33.115000+00:00,2024-01-14 09:29:36.887000+00:00,0.486373,0.236559,6,0.3,0.2,1,10,LOCAL,eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,admin,"[{""run_id"": ""8524d199f0214b47aa871f15c955e1de""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
2,4fa140fad4f64b29be2439ba4095536e,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/4fa140fad...,2024-01-14 09:29:28.911000+00:00,2024-01-14 09:29:33.087000+00:00,0.486373,0.236559,4,0.3,0.2,1,10,LOCAL,eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,admin,"[{""run_id"": ""4fa140fad4f64b29be2439ba4095536e""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
3,7f70ce5ceca04fa3b966e5ac243ee858,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/7f70ce5ce...,2024-01-14 09:29:25.298000+00:00,2024-01-14 09:29:28.889000+00:00,0.486373,0.236559,2,0.3,0.2,1,10,LOCAL,eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,admin,"[{""run_id"": ""7f70ce5ceca04fa3b966e5ac243ee858""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
4,fe31c784563f4ca58fac5088bb6a4a81,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/fe31c7845...,2024-01-14 09:29:21.633000+00:00,2024-01-14 09:29:25.274000+00:00,0.475191,0.225806,8,0.2,0.2,1,10,LOCAL,eta_0.2_subsample_1_max_depth_10_gamma_0.2_max...,admin,"[{""run_id"": ""fe31c784563f4ca58fac5088bb6a4a81""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,04172505f18e4c72979f38f65811133f,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/04172505f...,2024-01-14 08:54:59.589000+00:00,2024-01-14 08:55:02.899000+00:00,0.558416,0.311828,2,0.1,0.01,0.5,6,LOCAL,eta_0.01_subsample_0.5_max_depth_6_gamma_0.1_m...,admin,"[{""run_id"": ""04172505f18e4c72979f38f65811133f""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
572,463b8351805544d4b3dfc06b11ef426c,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/463b83518...,2024-01-14 08:54:55.982000+00:00,2024-01-14 08:54:59.554000+00:00,0.558416,0.311828,8,0.01,0.01,0.5,6,LOCAL,eta_0.01_subsample_0.5_max_depth_6_gamma_0.01_...,admin,"[{""run_id"": ""463b8351805544d4b3dfc06b11ef426c""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
573,2d3f87cbf0194935ad0a336d46945974,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/2d3f87cbf...,2024-01-14 08:54:52.506000+00:00,2024-01-14 08:54:55.965000+00:00,0.558416,0.311828,6,0.01,0.01,0.5,6,LOCAL,eta_0.01_subsample_0.5_max_depth_6_gamma_0.01_...,admin,"[{""run_id"": ""2d3f87cbf0194935ad0a336d46945974""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
574,e76fae5d1bc54c76bdbdf0fe5b7c4381,880873042180386125,FINISHED,mlflow-artifacts:/880873042180386125/e76fae5d1...,2024-01-14 08:54:49.003000+00:00,2024-01-14 08:54:52.485000+00:00,0.558416,0.311828,4,0.01,0.01,0.5,6,LOCAL,eta_0.01_subsample_0.5_max_depth_6_gamma_0.01_...,admin,"[{""run_id"": ""e76fae5d1bc54c76bdbdf0fe5b7c4381""...",C:\Users\admin\anaconda3\envs\mlopset\lib\site...
